In [63]:
import matlab.engine
import os
import csv
import sys
import time
import soundfile as sf
from datetime import timedelta
sys.path.append("../utils")
sys.path.append("../models/NMF")
from xml_interface import training_files_and_mix, onset_times
from measures import f_measure
import numpy as np

In [8]:
eng = matlab.engine.start_matlab()

In [64]:
audio_folder = r"C:/Cambridge/3rd Year/dissertation/IDMT-SMT-DRUMS-V2/audio/"
annotation_folder = r"C:/Cambridge/3rd Year/dissertation/IDMT-SMT-DRUMS-V2/annotation_xml/"

xml_files = [f for f in os.listdir(annotation_folder) if f.endswith('.xml')]
xml_files_sample = xml_files[:5]

In [71]:
# Input: XML file.
# Output: F-measure, compute time, real time.
def run_matlab_nmf(xml_file):
    try: 
        start_time = time.monotonic()
        loc = training_files_and_mix(annotation_folder + xml_file)[3]
        (hh_onsets, sd_onsets, kd_onsets) = onset_times(annotation_folder + xml_file)
        # get onsets from matlab function
        [hh,bd,sd] = eng.NmfDrum(audio_folder + loc, nargout=3)
        windows = [0.06, 0.055, 0.05, 0.045, 0.04, 0.035, 0.03, 0.025, 0.02, 0.015, 0.01]
        f_measures = []
        # get f-measure for each tolerance window
        for window in windows:
            f_score, precision, recall = f_measure([np.asarray(hh), np.asarray(bd), np.asarray(sd)], hh_onsets, kd_onsets, sd_onsets, window)
            f_measures.append(f_score)
        end_time = time.monotonic()
        compute_time = timedelta(seconds=end_time - start_time).total_seconds()

        # get real time
        (x, fs) = sf.read(audio_folder + loc)  
        timeLen = len(x) / fs

        return f_measures, compute_time, timeLen
    except Exception as e:
        print("Error with file: " + xml_file)
        print(e)
        return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0, 0


In [73]:

with open('../results/matlab_results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['File Name','F-Scores','Compute Time', 'Real Time'])
    count = 1
    for xml_file in xml_files:
        f_measures, compute_time, timeLen = run_matlab_nmf(xml_file)
        print("File %(count)d of %(total)d, F-score: %(f_score).3f, Compute Time: %(compute_time)f\n" % 
                    {"count": count, "total": len(xml_files),"f_score": f_measures[2], "compute_time": compute_time})
        writer.writerow([xml_file, f_measures, compute_time, timeLen])
        count += 1

Error with file: RealDrum01_00#MIX.xml

  File C:\Cambridge\3rd Year\MATLAB\toolbox\matlab\audiovideo\audioread.m, line 167, in readaudio

  File C:\Cambridge\3rd Year\MATLAB\toolbox\matlab\audiovideo\audioread.m, line 160, in audioread

  File C:\Cambridge\3rd Year\dissertation\Testing NMF Toolkit\NmfDrumToolbox\src\NmfDrum.m, line 46, in NmfDrum
The filename specified was not found in the MATLAB path.

File 1 of 95, F-score: 0.000, Compute Time: 0.000000

File 2 of 95, F-score: 0.779, Compute Time: 11.734000

File 3 of 95, F-score: 0.784, Compute Time: 9.828000

File 4 of 95, F-score: 0.818, Compute Time: 9.109000

File 5 of 95, F-score: 0.932, Compute Time: 8.391000

File 6 of 95, F-score: 0.841, Compute Time: 8.157000

File 7 of 95, F-score: 0.904, Compute Time: 10.890000

File 8 of 95, F-score: 0.827, Compute Time: 11.047000

File 9 of 95, F-score: 0.980, Compute Time: 9.031000

File 10 of 95, F-score: 0.722, Compute Time: 10.156000

File 11 of 95, F-score: 0.704, Compute Time: 15